In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from scipy.interpolate import interp1d
import os
import time
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import warnings

# 경고 메시지 숨김 (clean output)
warnings.simplefilter(action='ignore', category=FutureWarning)

# === 설정 ===
TARGET_YEARS = range(2018, 2025)  # 2018 ~ 2024
MAX_ROUNDS = 24                   # F1 최대 라운드 수
OUTPUT_DIR = "f1_processed_warehouse" # 저장될 루트 폴더

# === [1] 라운드별 처리 함수 (Worker) ===
def process_single_round(args):
    year, round_num = args

    # MongoDB 연결 (프로세스별 개별 연결 필요)
    client = MongoClient("mongodb://localhost:27017/")
    db = client["f1_bigdata_db"]
    col_telemetry = db["telemetry_raw"]

    try:
        # 1. 해당 라운드 데이터만 인출
        projection = {
            "_id": 0,
            "Year": 1, "Round": 1, "Driver": 1, "LapNumber": 1,
            "Speed": 1, "RPM": 1, "nGear": 1, "Throttle": 1, "Brake": 1, "DRS": 1,
            "Distance": 1, "Time": 1,
            "Compound": 1, "TyreLife": 1
        }

        cursor = col_telemetry.find({"Year": year, "Round": round_num}, projection)
        df_raw = pd.DataFrame(list(cursor))

        if df_raw.empty:
            return None # 데이터 없으면 스킵

        # 2. 기본 전처리
        df_raw.dropna(subset=['Speed', 'Distance', 'LapNumber'], inplace=True)
        df_raw['nGear'] = df_raw['nGear'].fillna(0).astype(int)
        df_raw['Brake'] = df_raw['Brake'].astype(int)
        df_raw['DRS'] = df_raw['DRS'].fillna(0).astype(int)

        # 3. 데이터 보간 (Resampling)
        processed_laps = []
        grouped = df_raw.groupby(['Driver', 'LapNumber']) # 이미 Year/Round는 필터링됨

        for (driver, lap_num), group in grouped:
            try:
                # 중복 제거 및 유효성 검사
                group = group.drop_duplicates(subset=['Distance'])
                if len(group) < 10 or group['Distance'].max() < 2000:
                    continue

                # 보간 기준점 (10m 단위)
                max_dist = group['Distance'].max()
                new_dist = np.arange(0, max_dist, 10)

                # 보간 함수 생성
                # kind='linear'가 속도/정확도 밸런스가 가장 좋음
                f_speed = interp1d(group['Distance'], group['Speed'], kind='linear', fill_value="extrapolate")
                f_rpm = interp1d(group['Distance'], group['RPM'], kind='linear', fill_value="extrapolate")
                f_throttle = interp1d(group['Distance'], group['Throttle'], kind='linear', fill_value="extrapolate")
                f_brake = interp1d(group['Distance'], group['Brake'], kind='nearest', fill_value="extrapolate")
                f_gear = interp1d(group['Distance'], group['nGear'], kind='nearest', fill_value="extrapolate")
                f_drs = interp1d(group['Distance'], group['DRS'], kind='nearest', fill_value="extrapolate")

                # DataFrame 생성
                df_lap = pd.DataFrame({
                    'Distance': new_dist,
                    'Speed': f_speed(new_dist),
                    'RPM': f_rpm(new_dist),
                    'Throttle': f_throttle(new_dist),
                    'Brake': f_brake(new_dist),
                    'nGear': f_gear(new_dist),
                    'DRS': f_drs(new_dist)
                })

                # 메타데이터 할당
                df_lap['Year'] = int(year)
                df_lap['Round'] = int(round_num)
                df_lap['Driver'] = int(driver) if str(driver).isdigit() else driver
                df_lap['LapNumber'] = int(lap_num)

                # 타이어 정보
                compound = group['Compound'].iloc[0] if 'Compound' in group.columns else 'UNKNOWN'
                tyre_life = group['TyreLife'].iloc[0] if 'TyreLife' in group.columns else 0
                df_lap['Compound'] = compound
                df_lap['TyreLife'] = tyre_life

                # LapTime 계산
                lap_time = group['Time'].max() - group['Time'].min()
                # Time 컬럼이 float(초)인지 datetime인지 확인 필요. 보통 float 가정
                # 만약 에러나면 여기 확인 필요
                df_lap['LapTime_Sec'] = float(lap_time)

                processed_laps.append(df_lap)

            except Exception:
                continue

        if not processed_laps:
            return None

        # 병합 및 저장
        df_final = pd.concat(processed_laps, ignore_index=True)

        # 데이터 타입 최적화 (용량 절약)
        df_final['Speed'] = df_final['Speed'].astype('float32')
        df_final['RPM'] = df_final['RPM'].astype('float32')
        df_final['Throttle'] = df_final['Throttle'].astype('float32')
        df_final['Distance'] = df_final['Distance'].astype('float32')

        # Parquet Partition 저장 (Year/Round 폴더 자동 생성)
        save_path = f"{OUTPUT_DIR}/year={year}/round={round_num}"
        os.makedirs(save_path, exist_ok=True)
        file_name = f"{save_path}/telemetry.parquet"

        # fastparquet 엔진 사용 (없으면 pyarrow 자동 사용)
        df_final.to_parquet(file_name, index=False, compression='snappy')

        return f"✅ {year}-R{round_num}: 저장 완료 ({len(df_final):,} rows)"

    except Exception as e:
        return f"❌ {year}-R{round_num} 에러: {str(e)}"
    finally:
        client.close()


In [ ]:

# === [2] 메인 실행부 (병렬 처리) ===
if __name__ == "__main__":
    start_total = time.time()

    # 작업 목록 생성
    tasks = [(y, r) for y in TARGET_YEARS for r in range(1, MAX_ROUNDS + 1)]

    print(f"🚀 F1 데이터 전처리 시작 ({TARGET_YEARS[0]} ~ {TARGET_YEARS[-1]})")
    print(f"📂 저장 경로: ./{OUTPUT_DIR}/...")
    print(f"🔢 총 작업 수: {len(tasks)} 개 라운드")

    # CPU 코어 수에 맞춰 병렬 처리
    max_workers = max(1, os.cpu_count() - 2)  # 여유 코어 2개 남김

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # tqdm으로 진행바 표시
        results = list(tqdm(executor.map(process_single_round, tasks), total=len(tasks), desc="Processing"))

    # 결과 요약 출력
    success_count = sum(1 for r in results if r and "✅" in r)
    print(f"\n🏁 작업 완료! 성공: {success_count} / {len(tasks)}")
    print(f"⏱️ 총 소요 시간: {(time.time() - start_total)/60:.1f}분")
    print(f"💡 데이터 로드 방법: pd.read_parquet('{OUTPUT_DIR}')")

In [3]:
# 병렬 처리(Executor) 부분을 일반 for문으로 변경한 디버깅용 코드
if __name__ == "__main__":
    start_total = time.time()

    tasks = [(y, r) for y in TARGET_YEARS for r in range(1, MAX_ROUNDS + 1)]

    print(f"🚀 F1 데이터 전처리 시작 ({TARGET_YEARS[0]} ~ {TARGET_YEARS[-1]}) - 순차 처리 모드")
    print(f"📂 저장 경로: ./{OUTPUT_DIR}/...")

    results = []
    # tqdm으로 감싸서 진행 상황 보기
    for task in tqdm(tasks, desc="Processing"):
        # 병렬이 아닌 직접 호출
        res = process_single_round(task)
        results.append(res)

    success_count = sum(1 for r in results if r and "✅" in r)
    print(f"\n🏁 작업 완료! 성공: {success_count} / {len(tasks)}")
    print(f"⏱️ 총 소요 시간: {(time.time() - start_total)/60:.1f}분")

🚀 F1 데이터 전처리 시작 (2018 ~ 2024) - 순차 처리 모드
📂 저장 경로: ./f1_processed_warehouse/...


Processing: 100%|██████████| 168/168 [2:05:56<00:00, 44.98s/it] 


🏁 작업 완료! 성공: 148 / 168
⏱️ 총 소요 시간: 125.9분


Training Dataset 생성

In [3]:
import pandas as pd
import numpy as np

# 설정
INPUT_DIR = "f1_processed_warehouse"
OUTPUT_FILE = "f1_training_dataset_v2.parquet"

def create_clean_dataset():
    print("📂 데이터 재로딩 및 정밀 필터링 시작...")

    # [수정 포인트] 불러올 때는 '원본 컬럼명'을 써야 합니다. (_mean 제거)
    cols = [
        'Year', 'Round', 'Driver', 'LapNumber',
        'Speed', 'Throttle', 'Brake', 'TyreLife', 'Compound', 'LapTime_Sec'
    ]

    # 1. 데이터 로드
    try:
        df = pd.read_parquet(INPUT_DIR, columns=cols)
        print(f"✅ 로드 완료: {len(df):,} 행 (Raw Telemetry)")
    except Exception as e:
        print(f"❌ 로드 중 에러 발생: {e}")
        return

    # 2. 랩 단위 요약 (Aggregating)
    print("⚙️ 랩 단위 데이터 압축 및 요약 중...")
    grouped = df.groupby(['Year', 'Round', 'Driver', 'LapNumber'])

    train_df = grouped.agg({
        'Speed': 'mean',
        'Throttle': 'mean',
        'Brake': 'mean',
        'TyreLife': 'max',
        'Compound': 'first',
        'LapTime_Sec': 'max'
    }).reset_index()

    # 컬럼명 변경 (여기서 _mean을 붙여줍니다)
    train_df.columns = [
        'Year', 'Round', 'Driver', 'LapNumber',
        'Speed_mean', 'Throttle_mean', 'Brake_mean',
        'TyreLife_max', 'Compound', 'LapTime_Sec'
    ]

    # === [핵심] 3. 이상치 제거 (107% 룰 적용) ===
    print("🧹 비정상 랩(피트스탑, SC, 아웃랩) 제거 중...")

    clean_laps = []

    # 라운드별 그룹화하여 107% 컷오프 적용
    # (tqdm을 쓰면 좋지만 dependency 문제 방지를 위해 생략하거나 간단히 루프)
    for (year, round_num), group in train_df.groupby(['Year', 'Round']):
        # 해당 라운드의 전체 랩 중 가장 빠른 기록 찾기
        fastest_lap = group['LapTime_Sec'].min()

        # 107% 룰: 1등 기록보다 7% 이상 느리면 정상적인 레이스 랩이 아님 (피트인/사고 등)
        threshold = fastest_lap * 1.07

        # 필터링: 너무 느린 랩(threshold 초과) & 너무 빠른 오류 랩(50초 미만) 제거
        valid_laps = group[
            (group['LapTime_Sec'] <= threshold) &
            (group['LapTime_Sec'] > 50)
        ]
        clean_laps.append(valid_laps)

    if not clean_laps:
        print("❌ 유효한 랩이 하나도 없습니다. 기준이 너무 엄격할 수 있습니다.")
        return

    train_df = pd.concat(clean_laps)

    # 4. 타이어 인코딩
    compound_map = {
        'SOFT': 1, 'MEDIUM': 2, 'HARD': 3,
        'INTERMEDIATE': 4, 'WET': 5, 'UNKNOWN': 0
    }
    # 대소문자 무시 처리
    train_df['Compound_Encoded'] = train_df['Compound'].astype(str).str.upper().map(compound_map).fillna(0).astype(int)

    # 타이어 정보 없는 데이터 삭제
    train_df = train_df[train_df['Compound_Encoded'] != 0]

    print(f"✅ 정제 완료: {len(train_df):,} 랩 (진짜 레이싱 랩만 남김)")

    # 저장
    train_df.to_parquet(OUTPUT_FILE, index=False)
    print(f"💾 저장 완료: {OUTPUT_FILE}")

if __name__ == "__main__":
    create_clean_dataset()

📂 데이터 재로딩 및 정밀 필터링 시작...
✅ 로드 완료: 80,378,016 행 (Raw Telemetry)
⚙️ 랩 단위 데이터 압축 및 요약 중...
🧹 비정상 랩(피트스탑, SC, 아웃랩) 제거 중...
✅ 정제 완료: 73,241 랩 (진짜 레이싱 랩만 남김)
💾 저장 완료: f1_training_dataset_v2.parquet
